# Distributed

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/intro_ml/blob/main/docs/05_distributed.ipynb)

```{note}
If you’re in COLAB or have a local CUDA GPU, you can follow along with the more computationally intensive training in this lesson.

For those in COLAB, ensure the session is using a GPU by going to: Runtime > Change runtime type > Hardware accelerator = GPU.
```

In [ ]:
# if you're using colab, then install the required modules
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install --quiet --upgrade ray

Distributing training over multiple devices generally uses either:

- [Data parallelism](https://developers.google.com/machine-learning/glossary/#data-parallelism)
    - _Split the **data** over multiple devices._
    - Single model copied to multiple devices.
    - Useful for big data.
    - Simpler.
- [Model parallelism](https://developers.google.com/machine-learning/glossary/#model-parallelism)
    - _Split the **model** over multiple devices._
    - Single data copied to multiple devices.
    - Can be useful for big models (for some architectures).
    - More complex.
    
This lesson focuses on data parallelism.

## [Ray Train](https://docs.ray.io/en/latest/train/train.html)

Ray Train is a useful tool for distributed deep learning training for [TensorFlow (Keras)](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras) and [PyTorch](https://pytorch.org/tutorials/beginner/dist_overview.html).

It handles the set up for you (e.g., [`TF_CONFIG`](https://www.tensorflow.org/guide/distributed_training#setting_up_the_tf_config_environment_variable) in TensorFlow).

There are a range of examples [here](https://docs.ray.io/en/latest/train/examples.html).

```{warning}
Note, Ray doesn't currently work on POWER9 machines e.g., Bede. See, [GitHub issue](https://github.com/ray-project/ray/issues/7476).
```

### Example: [TensorFlow (Keras) MNIST](https://docs.ray.io/en/latest/train/examples/tensorflow_mnist_example.html)

In [ ]:
import argparse
import json
import os

import numpy as np
import ray
import tensorflow as tf
from ray.train import Trainer
from tensorflow.keras.callbacks import Callback

#### [Define callback for reporting](https://docs.ray.io/en/latest/train/user_guide.html#logging-monitoring-and-callbacks)

In [ ]:
class TrainReportCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        ray.train.report(**logs)

#### Set up the dataset and model

The dataset will be split (sharded) across the workers.

```{tip}
The default [auto-sharding](https://www.tensorflow.org/tutorials/distribute/input#sharding) by `FILE` can cause warning messages if the data is in one file. Instead, you can specify to auto-shard by data using: `tf.data.experimental.AutoShardPolicy.DATA` (which it will fall back to anyway).
```

In [ ]:
def mnist_dataset(batch_size):
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    # The `x` arrays are in uint8 and have values in the [0, 255] range.
    # You need to convert them to float32 with values in the [0, 1] range.
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    ds_train = (
        tf.data.Dataset.from_tensor_slices((x_train, y_train))
        .shuffle(60000)
        .repeat()
        .batch(batch_size)
    )

    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = (
        tf.data.experimental.AutoShardPolicy.DATA
    )
    ds_train = ds_train.with_options(options)

    return ds_train

When building the model, a `config` keyword argument is added to get different options such as the learning rate.

In [ ]:
def build_and_compile_cnn_model(config):
    learning_rate = config.get("lr", 0.001)
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(28, 28)),
            tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(10),
        ]
    )
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
        metrics=["accuracy"],
    )
    return model

#### Set up the training function for a _single_ worker

This also uses the `config` argument.

In [ ]:
def train_func(config):
    batch_size = 64
    single_worker_dataset = mnist_dataset(batch_size)
    single_worker_model = build_and_compile_cnn_model(config)
    single_worker_model.fit(
        single_worker_dataset,
        epochs=config["epochs"],
        steps_per_epoch=70,
        verbose=False,
    )

Now, you [configure training](https://docs.ray.io/en/latest/train/user_guide.html#configuring-training) using the `config` parameter.

In [ ]:
config = {"epochs": 3}

In [ ]:
train_func(config)

#### [Update training function](https://docs.ray.io/en/latest/train/user_guide.html#update-training-function)

1. Set the _global_ batch size
    - Each worker will process the same size batch as in the single-worker code.
2. Choose your TensorFlow distributed training strategy.
    - In this example we use the [MultiWorkerMirroredStrategy](https://www.tensorflow.org/guide/distributed_training#multiworkermirroredstrategy) for synchronous training of multiple workers across many machines.
        - For multiple workers on _one_ machine, use [MirroredStrategy](https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy).
        - In general, the mirrored strategy mirrors the parameters across the workers, ensuring replicas are identical.
    - Within the strategy scope context manager, you build and compile the model.

In [ ]:
def train_func(config):
    per_worker_batch_size = config.get("batch_size", 64)
    epochs = config.get("epochs", 3)
    steps_per_epoch = config.get("steps_per_epoch", 70)

    tf_config = json.loads(os.environ["TF_CONFIG"])
    num_workers = len(tf_config["cluster"]["worker"])

    strategy = tf.distribute.MultiWorkerMirroredStrategy()

    global_batch_size = per_worker_batch_size * num_workers
    multi_worker_dataset = mnist_dataset(global_batch_size)

    with strategy.scope():
        # model building/compiling need to be within strategy.scope()
        multi_worker_model = build_and_compile_cnn_model(config)

    history = multi_worker_model.fit(
        multi_worker_dataset,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        callbacks=[TrainReportCallback()],
        verbose=False,
    )
    results = history.history
    return results

#### [Create Ray Train Trainer](https://docs.ray.io/en/latest/train/user_guide.html#create-ray-train-trainer)

The `Trainer` manages state and training.

Ray Train enables different `backend` options e.g., `tensorflow`.

In [ ]:
def train_tensorflow_mnist(num_workers=1, use_gpu=False, epochs=4):
    trainer = Trainer(backend="tensorflow", num_workers=num_workers, use_gpu=use_gpu)
    trainer.start()
    results = trainer.run(
        train_func=train_func, config={"lr": 1e-3, "batch_size": 64, "epochs": epochs}
    )
    trainer.shutdown()
    print(f"Results: {results[0]}")

#### [Run the training](https://docs.ray.io/en/latest/train/user_guide.html#run-training-function)

Initialise and shutdown the Ray client:

In [ ]:
# ray.init()

In [ ]:
# cpu
# train_tensorflow_mnist()

# gpu
# train_tensorflow_mnist(use_gpu=True)

In [ ]:
# ray.shutdown()

#### Submit the job to HPC

This Python script is in full [here](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/tensorflow_ray_train_mnist_example.py).

##### CPUs

An example job submission script is (also [here](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/distributed_ml_on_arc4_cpu.bash)):

```bash
#!/bin/bash
#$ -cwd -V
#$ -l h_rt=00:30:00
#$ -pe smp 12
#$ -l h_vmem=6G

# activate conda and add to library path
conda activate intro_ml
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib

# run the CPU script
python tensorflow_ray_train_mnist_example.py --num-workers 12 --epochs 100
```

```{warning}
Sometimes the `LD_LIBRARY_PATH` variable will not include the path to the conda environment, resulting it in being unable to find some libraries e.g., TensorFlow, cuDNN.

To resolve this, in your script append the path to your activated conda environment to the `LD_LIBRARY_PATH` variable using:

`export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib`  
```

```{note}
These scripts are based on a personal miniconda with the environment activated at the terminal and the terminal `LD_LIBRARY_PATH` updated.  

There are other options e.g., using the [Anaconda](https://arcdocs.leeds.ac.uk/software/compilers/anaconda.html) distribution installed on ARC.  
```

To change the number of workers (either CPUs or GPUs), set the `--num-workers` argument.

Ensure that the scheduler request matches this number e.g.,:

- For 12 CPU workers, use `--num-workers 12` and `#$ -pe smp 12`.
- For 2 GPU workers, use `--num-workers 2` and `#$ -l coproc_v100=2`.
    - To use GPUs, you will also need the `--use-gpu True` flag.

In this simple example using 12 CPUs, the job efficiency was (using `qacct -j <JOBID>`):

```
Efficiency = 100 * cpu / (ru_wallclock * slots)
Efficiency = 100 * 10214 / (928 * 12)
Efficiency = 92 %
```

92% is good.

##### GPU(s)

```{note}
The tests below were for a [NVIDIA V100 with 32 GB of memory](https://arcdocs.leeds.ac.uk/usage/gpgpu.html#arc4).
```

An example job submission script is (also [here](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/distributed_ml_on_arc4_gpu.bash)):

```bash
#!/bin/bash
#$ -cwd -V
#$ -l h_rt=00:15:00
#$ -l coproc_v100=1

# activate conda and add to library path
conda activate swd8_intro_ml 
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib

# start the efficiency log for the GPU
nvidia-smi dmon -d 10 -s um -i 0 > efficiency_log &

# run the GPU script
python tensorflow_ray_train_mnist_example.py --use-gpu True --num-workers 1 --epochs 100

# stop the efficiency log
kill %1
```

You can check the efficiency of your GPU job using the [NVIDIA System Management Interface](https://developer.nvidia.com/nvidia-system-management-interface) i.e., `nvidia-smi`.

- `dmon` is for device monitoring.
- `-d X` records the metrics every X seconds.
- `-s um` is the metrics to sample, where u = utilisation and m = memory.
- `-i 0` is the device number, where 0 = first gpu. For multiple devices, add them separated by commas e.g., `-i 0,1`.

Then you can view the results from the `efficiency_log`:

```bash
# gpu    sm   mem   enc   dec    fb  bar1
# Idx     %     %     %     %    MB    MB
    0     0     0     0     0     0     2
    0     0     0     0     0     0     2
    0     0     0     0     0 31880     5
    0    34    15     0     0 31932     5
```

The main columns are `sm` (for utilisation %) and `mem` (for memory %).

Here, the job utilised approximately 35% of the GPU and 15% of its memory. (The first few rows are blank as the GPU was idle while the CPU downloaded the data.)

That's quite low, though this was for a simple MNIST example.

A more complex problem will likely utilise the GPU better.

### Example: [TensorFlow (Keras) transfer learning](tf_transfer_learning)

In lesson 4, we looked at a TensorFlow (Keras) transfer learning example. To use this script with Ray Train, you primarily need to create the dataset pipeline and model creation functions. You can use the examples here as a foundation. The full script for this example is [here](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/tensorflow_ray_train_transfer_learning_example.py).

Running this code on a single GPU utilised approximately 90% of the GPU and 45% of its memory.

This is good.

Multiple GPUs is not needed here, though they could be requested by simply changing:  

- Ray Train flag `--num-workers 2`
- Scheduler option `#$ -l coproc_v100=2`
- GPU monitoring `nvidia-smi dmon -i 0,1`

```{tip}
To utilise GPU(s) efficiently, try to match the resources requested to what your problem requires e.g., the model of GPU, the memory it has, etc.
```

## [PyTorch (Lightning)](https://pytorch-lightning.readthedocs.io/en/stable/clouds/cluster.html)

PyTorch Lightning comes with great built-in functionality to scale to multiple GPUs.

First though, we should check that the problem requires these resources.

A [simple example for MNIST](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/pytorch_lightning_mnist_example.py) utilises about 8% of a single V100 GPU.

The [PyTorch Lightning transfer learning example](torch_transfer_learning) (lesson 4) utilises about 15% of a single V100 GPU (full script [here](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/pytorch_lightning_transfer_learning_example.py)).

To demonstrate running this transfer learning example over multiple GPUs, we could unfreeze the pre-trained model to increase the workload.

This is normally not recommended, though is useful for demonstration purposes here of a complex model.

To unfreeze the pre-trained model:

- Set `trainer.finetune(strategy="unfreeze")`.
- This is also the default setting.

Then the steps to run over multiple GPUs are:

1. Specify the [data parallel strategy](https://pytorch-lightning.readthedocs.io/en/stable/advanced/model_parallel.html#ddp-optimizations) in the Trainer:

    ```python
    from pytorch_lightning.strategies import DDPStrategy

    trainer = flash.Trainer(
        max_epochs=100,
        accelerator="gpu",
        devices=torch.cuda.device_count(),
        strategy=DDPStrategy(find_unused_parameters=False),
    )
    ```

    - Note, setting `find_unused_parameters=False` [can improve performance](https://pytorch.org/docs/stable/notes/ddp.html#internal-design).


2. Set the scheduler to use multiple GPUs:

    - e.g., `#$ -l coproc_v100=2`.
    


3. Adding GPU monitoring for multiple GPUs:

    - e.g., `nvidia-smi dmon -i 0,1`.


This utilised 2 V100 GPUs at approximately 40%. This isn't great, though it demonstrates the ease of distributed training with PyTorch Lightning.

## Jupyter Notebook to HPC

Once you've finished testing out different ideas locally in a Jupyter Notebook, you can then convert this to an executable script to run on a HPC.

This is because the HPCs (currently accessible at Leeds, at least) are suitable for non-interactive batch jobs.

The steps are to:

- [Clean non-essential code](clean_nonessential_code)
- [Refactor Jupyter Notebook code into functions](refactor_into_functions)
- [Create a Python script](create_python_script)
- [Create submission script](create_submission_script)
- [(Optional) Create unit tests](create_unit_tests)

(clean_nonessential_code)=
### Clean non-essential code

Some code added during the experimentation phase was only needed to test out ideas and explore the data.

This non-essential code can be removed to make it more maintainable and performant.

Let's use the example from the [TensorFlow Datasets MNIST example](tensorflow_datasets) we saw in Lesson 3.

In [ ]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

# global setup
tf.keras.utils.set_random_seed(42)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
AUTOTUNE = tf.data.AUTOTUNE
NUM_EPOCHS = 5

# download the data
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(ds_train)
print(ds_info)

# preprocess the data
def normalise_image(image, label):
    return tf.cast(image, tf.float32) / 255.0, label


# create data pipelines
def training_pipeline(ds_train):
    ds_train = ds_train.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
    ds_train = ds_train.batch(128)
    ds_train = ds_train.prefetch(AUTOTUNE)
    return ds_train


def test_pipeline(ds_test):
    ds_test = ds_test.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_test = ds_test.batch(128)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(AUTOTUNE)
    return ds_test


ds_train = training_pipeline(ds_train)
ds_val = training_pipeline(ds_val)
ds_test = test_pipeline(ds_test)


# create the model
inputs = tf.keras.Input(shape=(28, 28, 1), name="inputs")
x = tf.keras.layers.Flatten(name="flatten")(inputs)
x = tf.keras.layers.Dense(128, activation="relu", name="layer1")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation="relu", name="layer2")(x)
outputs = tf.keras.layers.Dense(10, name="outputs")(x)

model = tf.keras.Model(inputs, outputs, name="functional")

# view the model
model.summary()

# compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
)

# train the model
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=NUM_EPOCHS,
    verbose=False,
)

# plot the model accuracy
epochs_range = range(1, NUM_EPOCHS + 1)

plt.plot(epochs_range, history.history["accuracy"], "bo", label="Training accuracy")
plt.plot(
    epochs_range, history.history["val_accuracy"], "b", label="Validation accuracy"
)
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim([0.9, 1.0])
plt.legend()
plt.show()

# save the model
path_models = f"{os.getcwd()}/models"
model.save(f"{path_models}/model_tf_mnist")

We can now remove the non-essential code, as the experimentation phase is complete.

Here, this included:

- Removing data information from the download.
- Removing unrequired print statements.
- Viewing model summary.
- Removing/replacing plots with text output.

In [ ]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

# global setup
tf.keras.utils.set_random_seed(42)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
AUTOTUNE = tf.data.AUTOTUNE
NUM_EPOCHS = 5

# download the data
(ds_train, ds_val, ds_test) = tfds.load(
    "mnist",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    shuffle_files=True,
    as_supervised=True,
    with_info=False,
)

# preprocess the data
def normalise_image(image, label):
    return tf.cast(image, tf.float32) / 255.0, label


# create data pipelines
def training_pipeline(ds_train):
    ds_train = ds_train.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
    ds_train = ds_train.batch(128)
    ds_train = ds_train.prefetch(AUTOTUNE)
    return ds_train


def test_pipeline(ds_test):
    ds_test = ds_test.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_test = ds_test.batch(128)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(AUTOTUNE)
    return ds_test


ds_train = training_pipeline(ds_train)
ds_val = training_pipeline(ds_val)
ds_test = test_pipeline(ds_test)


# create the model
inputs = tf.keras.Input(shape=(28, 28, 1), name="inputs")
x = tf.keras.layers.Flatten(name="flatten")(inputs)
x = tf.keras.layers.Dense(128, activation="relu", name="layer1")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation="relu", name="layer2")(x)
outputs = tf.keras.layers.Dense(10, name="outputs")(x)

model = tf.keras.Model(inputs, outputs, name="functional")

# compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
)

# train the model
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=NUM_EPOCHS,
    verbose=False,
)

# view the model accuracy
print(f"Training accuracy: {[round(num, 2) for num in history.history['accuracy']]}")
print(
    f"Validation accuracy: {[round(num, 2) for num in history.history['val_accuracy']]}"
)

# save the model
path_models = f"{os.getcwd()}/models"
model.save(f"{path_models}/model_tf_mnist")

(refactor_into_functions)=
### Refactor Jupyter Notebook code into functions

Now, any code that is not already in functions, can be refactored into functions.

Modularising the code like this helps with diagnosing errors and creating tests.

Here, we added functions for:

- Downloading the data.
- Creating and compiling the model.
- Training the model.
- Saving the model.
- A main function for the whole workflow.

In [ ]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

# global setup
tf.keras.utils.set_random_seed(42)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
AUTOTUNE = tf.data.AUTOTUNE
NUM_EPOCHS = 5

# download the data
def download_data():
    (ds_train, ds_val, ds_test) = tfds.load(
        "mnist",
        split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
        shuffle_files=True,
        as_supervised=True,
        with_info=False,
    )
    return ds_train, ds_val, ds_test


# preprocess the data
def normalise_image(image, label):
    return tf.cast(image, tf.float32) / 255.0, label


# create data pipelines
def training_pipeline(ds_train):
    ds_train = ds_train.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
    ds_train = ds_train.batch(128)
    ds_train = ds_train.prefetch(AUTOTUNE)
    return ds_train


def test_pipeline(ds_test):
    ds_test = ds_test.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_test = ds_test.batch(128)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(AUTOTUNE)
    return ds_test


# create and compile the model
def create_and_compile_model():
    inputs = tf.keras.Input(shape=(28, 28, 1), name="inputs")
    x = tf.keras.layers.Flatten(name="flatten")(inputs)
    x = tf.keras.layers.Dense(128, activation="relu", name="layer1")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation="relu", name="layer2")(x)
    outputs = tf.keras.layers.Dense(10, name="outputs")(x)

    model = tf.keras.Model(inputs, outputs, name="functional")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
    )

    return model


# train the model
def train_model():
    history = model.fit(
        ds_train,
        validation_data=ds_val,
        epochs=NUM_EPOCHS,
        verbose=False,
    )
    return history


# save the model
def save_model(model):
    path_models = f"{os.getcwd()}/models"
    model.save(f"{path_models}/model_tf_mnist")


# combine the functions in a call to main
def main():
    ds_train, ds_val, ds_test = download_data()

    ds_train = training_pipeline(ds_train)
    ds_val = training_pipeline(ds_val)
    ds_test = test_pipeline(ds_test)

    model = create_and_compile_model()
    history = train_model()
    save_model(model)


# run the functions
main()

# view the model accuracy
print(f"Training accuracy: {[round(num, 2) for num in history.history['accuracy']]}")
print(
    f"Validation accuracy: {[round(num, 2) for num in history.history['val_accuracy']]}"
)

(create_python_script)=
### Create a Python script

First, the call to `main()` should be placed inside a conditional invocation i.e.,:

```python
if __name__ == '__main__':
    main()
```

Now, the script can be called from a terminal by running `python script.py`.

Then, the rest of the code can be converted by either:

#### Option 1 (recommended)

Convert a _single cell_ to a script using the `%%writefile` IPython magic command.

In [ ]:
%%writefile jupyter-to-hpc_tf-mnist-example.py

import os

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

# global setup
tf.keras.utils.set_random_seed(42)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
AUTOTUNE = tf.data.AUTOTUNE
NUM_EPOCHS = 5

# download the data
def download_data():
    (ds_train, ds_val, ds_test) = tfds.load(
        "mnist",
        split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
        shuffle_files=True,
        as_supervised=True,
        with_info=False,
    )
    return ds_train, ds_val, ds_test


# preprocess the data
def normalise_image(image, label):
    return tf.cast(image, tf.float32) / 255.0, label


# create data pipelines
def training_pipeline(ds_train):
    ds_train = ds_train.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
    ds_train = ds_train.batch(128)
    ds_train = ds_train.prefetch(AUTOTUNE)
    return ds_train


def test_pipeline(ds_test):
    ds_test = ds_test.map(normalise_image, num_parallel_calls=AUTOTUNE)
    ds_test = ds_test.batch(128)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(AUTOTUNE)
    return ds_test


# create and compile the model
def create_and_compile_model():
    inputs = tf.keras.Input(shape=(28, 28, 1), name="inputs")
    x = tf.keras.layers.Flatten(name="flatten")(inputs)
    x = tf.keras.layers.Dense(128, activation="relu", name="layer1")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation="relu", name="layer2")(x)
    outputs = tf.keras.layers.Dense(10, name="outputs")(x)

    model = tf.keras.Model(inputs, outputs, name="functional")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
    )

    return model


# train the model
def train_model():
    history = model.fit(
        ds_train,
        validation_data=ds_val,
        epochs=NUM_EPOCHS,
        verbose=False,
    )
    return history


# save the model
def save_model(model):
    path_models = f"{os.getcwd()}/models"
    model.save(f"{path_models}/model_tf_mnist")


# combine the functions in a call to main
def main():
    ds_train, ds_val, ds_test = download_data()

    ds_train = training_pipeline(ds_train)
    ds_val = training_pipeline(ds_val)
    ds_test = test_pipeline(ds_test)

    model = create_and_compile_model()
    history = train_model()
    save_model(model)


if __name__ == "__main__":
    # run the functions
    main()

    # view the model accuracy
    print(
        f"Training accuracy: {[round(num, 2) for num in history.history['accuracy']]}"
    )
    print(
        f"Validation accuracy: {[round(num, 2) for num in history.history['val_accuracy']]}"
    )

#### Option 2

Convert the _entire_ notebook to a script using the `nbconvert` package:

```bash
jupyter nbconvert "my_notebook.ipynb" --to script
```

#### Option 3

Convert the _entire_ notebook to a script using the Jupyter menu.

From within the Jupyter Notebook, click `File` > `Save and Export Notebook As ...` > `Executable Script`.

This should convert the Jupyter Notebook to a `.py` file, and download it locally.

(create_submission_script)=
### Create submission script

For your HPC (e.g., ARC4), create the submission script.

For example:

```bash
#!/bin/bash
#$ -cwd
#$ -l h_rt=00:05:00
#$ -l coproc_v100=1

conda activate intro_ml 
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib  # (sometimes required)

python jupyter-to-hpc_tf-mnist-example.py
```

Ensure both of these files are on the HPC, alongside the corresponding conda environment.

(create_unit_tests)=
### (Optional) Create unit tests

It's good practise to write [tests](https://the-turing-way.netlify.app/reproducible-research/testing.html) for your code.

[pytest](https://docs.pytest.org/en/6.2.x/) are [NumPy](https://numpy.org/doc/stable/reference/testing.html) are both good, commonly-used tools for testing in Python.

There are various [levels, types, and methods](https://softwaretestingfundamentals.com/) of testing.

For example, you could [unit test](https://softwaretestingfundamentals.com/unit-testing/) individual functions, like the downloading and splitting of data:

```python
import numpy as np

def test_split_data(ds_train, ds_val, ds_test):
    num_samples_train = len(ds_train)
    num_samples_val = len(ds_val)
    num_samples_test = len(ds_test)
    
    num_samples = num_samples_train + num_samples_val + num_samples_test
    
    np.testing.assert_almost_equal(num_samples_train / num_samples, 0.8, decimal=3)
    np.testing.assert_almost_equal(num_samples_val / num_samples, 0.1, decimal=3)
    np.testing.assert_almost_equal(num_samples_test / num_samples, 0.1, decimal=3)
    

test_split_data(ds_train, ds_val, ds_test)
```

You could also perform [system testing](https://softwaretestingfundamentals.com/system-testing/). For example, checking the final validation accuracy is above a threshold:

```python
def test_final_val_accuracy_above_threshold(threshold):
    assert history.history['val_accuracy'][-1] >= threshold
    
test_final_val_accuracy_above_threshold(0.96)
```

## Questions

```{admonition} Question 1

What are the two ways to parallelise machine learning, and which way is simpler?

```

```{admonition} Question 2

How can you check the efficiency of a CPU job?

```

```{admonition} Question 3

How can you check the efficiency of a GPU job?

```

```{admonition} Question 4

What tools can help distribute TensorFlow and PyTorch code?

```

```{admonition} Question 5

In general, what should the batch size be for distributed work?

```

```{admonition} Question 6

What are some good steps for moving Jupyter Notebook code to HPC?

```

## {ref}`Solutions <distributed>`

## Key Points

```{important}

- [x] _Ensure that you really need to use distributed devices._
- [x] _Check everything first works on a single device._
- [x] _Use data parallelism (to split the data over multiple devices)._
- [x] _Take care when setting the global batch size._
- [x] _Check the efficiency of your jobs to ensure utilising the requested resources._
- [x] _When moving from Jupyter to HPC:_
    - [x] _Clean non-essential code._
    - [x] _Refactor Jupyter Notebook code into functions._
    - [x] _Create a Python script._
    - [x] _Create submission script._
    - [x] _Create tests._

```

## Further information

### Good practices

- Ensure everything works on a single device first, _before_ going distributed.
- Ensure that you need the overhead of distributing over multiple GPUs e.g., could you instead use 1 GPU and model checkpointing/transfer learning?
- Ensure that the problem is complex enough to utilise multiple GPUs efficiently.
- Batch the dataset with the global batch size e.g., for 8 devices each capable of a batch of 64 use the global batch size of 512 (= 8 * 64).  
- [Distributed training on PyTorch Lightning](https://pytorch-lightning.readthedocs.io/en/latest/advanced/model_parallel.html)
- Performance tips from [PyTorch](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html)

### Other options

- [Horovod](https://horovod.ai/)
    - A library to make distributed deep learning fast and easy to use.
- [DeepSpeed](https://www.deepspeed.ai/)
    - A deep learning optimization library that makes distributed training easy, efficient, and effective.
- [FairScale](https://fairscale.readthedocs.io/en/latest/)
    - A PyTorch extension library for high performance and large scale training.
 
### Resources

- [PyTorch Distributed Overview](https://pytorch.org/tutorials/beginner/dist_overview.html)
- [scikit-learn parallelism](https://scikit-learn.org/stable/computing/parallelism.html)
- [Convert ML experiment to production](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-convert-ml-experiment-to-production)